In [1]:
import numpy as np
from sklearn import *
import tensorflow as tf

<font size="5">data preprocessing</font>

In [2]:
data = np.genfromtxt("data/train.csv", delimiter=",", skip_header=1)
np.random.shuffle(data)
print(data.shape)

(42000, 785)


In [3]:
Xtest = np.genfromtxt("data/test.csv", delimiter=",", skip_header=1)
print(Xtest.shape)

(28000, 784)


In [4]:
X = data[:,1:]
Y = data[:,0]

print(X.shape)
print(Y.shape)


(42000, 784)
(42000,)


In [5]:
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2,3,4,5,6,7,8,9])
Y = lb.transform(Y)

<font size="5">feature selection</font>

In [6]:
vt = feature_selection.VarianceThreshold(10)
X = vt.fit_transform(X)
print(X.shape)

(42000, 612)


In [7]:
mask = vt.get_support()
Xtest = Xtest[:,mask]
print(Xtest.shape)

(28000, 612)


<font size="5">training/validation split</font>

In [8]:
Xtr = X[0:32000]
Ytr = Y[0:32000]
Xva = X[32000:42000]
Yva = Y[32000:42000]

<font size="5">tensorflow setup</font>

In [9]:
batch_size = tf.placeholder(tf.int64)
test_batch_size = tf.placeholder(tf.int64)
BATCH_SIZE = 500

In [10]:
x = tf.placeholder(tf.float32, shape=[None,Xtr.shape[1]]) 
y = tf.placeholder(tf.float32, shape=[None,10])
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()

iterator = dataset.make_initializable_iterator()
x_batch, y_batch = iterator.get_next()

train_init_op = iterator.make_initializer(dataset)

In [11]:
dense = tf.layers.dense(inputs=x_batch, units=1000, activation=tf.nn.relu)

logits = tf.layers.dense(inputs=dense, units=10)

In [12]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_batch, logits=logits)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

accuracy = tf.metrics.accuracy(labels=tf.argmax(y_batch, 1), predictions=tf.argmax(logits, 1))
val_acc_metric = tf.metrics.accuracy(labels=tf.argmax(Yva, 1), predictions=tf.argmax(logits, 1))

<font size="5">Run the Model</font>

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

sess.run(train_init_op, feed_dict={ x: Xtr, y: Ytr, batch_size: BATCH_SIZE})

n_batch = Xtr.shape[0] // BATCH_SIZE
print("n_batch: ", n_batch)

for i in range(100):
    print('EPOCH', i)
    tot_acc = 0
    tot_loss = 0
    for _ in range(n_batch):
        _, accuracy_val, loss_val = sess.run([optimizer, accuracy, loss])
        tot_acc += accuracy_val[0]
        tot_loss += loss_val
    print("accuracy: ", tot_acc/n_batch)
    print("loss: ", loss_val/n_batch)
    
    print('DONE WITH EPOCH')
        


n_batch:  64
EPOCH 0
accuracy:  0.2999678768683225
loss:  0.011916416697204113
DONE WITH EPOCH
EPOCH 1
accuracy:  0.5778138879686594
loss:  0.0092272087931633
DONE WITH EPOCH
EPOCH 2
accuracy:  0.689081558957696
loss:  0.0043506622314453125
DONE WITH EPOCH
EPOCH 3
accuracy:  0.7472963538020849
loss:  0.002994139678776264
DONE WITH EPOCH
EPOCH 4
accuracy:  0.7827887311577797
loss:  0.0029591687489300966
DONE WITH EPOCH
EPOCH 5
accuracy:  0.8077303040772676
loss:  0.0039281053468585014
DONE WITH EPOCH
EPOCH 6
accuracy:  0.8262225175276399
loss:  0.0028127175755798817
DONE WITH EPOCH
EPOCH 7
accuracy:  0.8406429653987288
loss:  0.002533235587179661
DONE WITH EPOCH
EPOCH 8
accuracy:  0.8520123846828938
loss:  0.0020353724248707294
DONE WITH EPOCH
EPOCH 9
accuracy:  0.8610891159623861
loss:  0.00366804632358253
DONE WITH EPOCH
EPOCH 10
accuracy:  0.8689944474026561
loss:  0.0026623087469488382
DONE WITH EPOCH
EPOCH 11
accuracy:  0.8758693598210812
loss:  0.0019218374509364367
DONE WITH EPOC

accuracy:  0.9730449821799994
loss:  0.00015917659038677812
DONE WITH EPOCH
EPOCH 99
accuracy:  0.9732866520062089
loss:  0.00015368884487543255
DONE WITH EPOCH


In [14]:
sess.run(train_init_op, feed_dict={ x: Xtest, y:[[0 for _ in range(10)] for __ in range(28000)], batch_size: 28000})

pred_y=tf.argmax(tf.nn.softmax(logits),1)
predictions = pred_y.eval(session=sess)
print(np.array(predictions).shape)

(28000,)


In [15]:
f = open("output/kaggle.csv", "w")
f.write("ImageId,Label\n")
for i in range(len(predictions)):
    f.write(str(i+1) + "," + str(predictions[i]) + "\n")
f.close()

PermissionError: [Errno 13] Permission denied: 'output/kaggle.csv'